In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import the important libraries first**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px

**Lets read the csv file and start exploring**

In [3]:
players=pd.read_csv('../input/commonwealth-games-2022/commonwealth games 2022 - players participated.csv')
players.head()

# EDA On Participating Players Dataset


**Our point of interest are :-**
> *1. Which country participates in most sports?*

> *2. Male vs female athlete count overall*

> *3. Age Distribution of athletes*

*One athlete has age as null so lets change that, her real age is 20 yrs*

*Note:- These are total sports and not events. each sport wil have its own x number of events*

In [4]:
players[players['AGE'].isna()]
players["AGE"]=players["AGE"].fillna(20.0)
players['AGE']=players['AGE'].astype('int')
totalunique_sports=players.drop_duplicates(subset=['SPORT','GENDER','TEAM'])


In [5]:
menunique_sports=totalunique_sports[totalunique_sports['GENDER']=='Male']
womenunique_sports=totalunique_sports[totalunique_sports['GENDER']=='Female']

**Top 10 Participating countries for Mens sports**

In [6]:
fig,ax=plt.subplots(figsize=(12,6))
x=menunique_sports[['TEAM','SPORT']].groupby("TEAM")['SPORT'].count().sort_values(ascending=False).head(10).reset_index()
bars=ax.bar(x=x['TEAM'],height=x['SPORT'],color='royalblue')
ax.tick_params(bottom=False,left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')

for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.3,
      round(bar.get_height(), 1),
      color='royalblue',
      horizontalalignment='center',
      weight='bold'
  )
plt.title('Top 10 Participating Nations for Mens Sports In 2022',fontsize=15)

fig.tight_layout()

****Top 10 Participating countries for Womens sports****

In [7]:
fig,ax=plt.subplots(figsize=(12,6))
x=womenunique_sports[['TEAM','SPORT']].groupby("TEAM")['SPORT'].count().sort_values(ascending=False).head(10).reset_index()
bars=ax.bar(x=x['TEAM'],height=x['SPORT'],color='crimson')
ax.tick_params(bottom=False,left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')

for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.3,
      round(bar.get_height(), 1),
      horizontalalignment='center',
      color='crimson',
      weight='bold'
  )

plt.title('Top 10 Participating Nations for Womens Sports In 2022',fontsize=15)


fig.tight_layout()

**conclusion**


*Across the graphs it is clear that England and Australia paticipate in most sports*



****

# 

**Lets find out the age distribution of athletes**

*1. For male athletes*

In [8]:
male_players = players[players['GENDER'] == 'Male']
ax=sns.displot(data=male_players['AGE'],bins=20,height=6,aspect=2,color='olive',kde=True)

plt.title('Age Distribution For Male Athletes',fontsize=15)
plt.show()

*2. Female Athletes*

In [9]:
female_players = players[players['GENDER'] == 'Female']
ax=sns.displot(data=female_players['AGE'],bins=20,height=6,aspect=2,color='indianred',kde=True)
plt.title('Age Distribution For Female Athletes',fontsize=15)
plt.show()

**Conclusion**

*Age of majority of the Athletes are above 17 and below 35*

*There are outliers where age is above 50*


*After digging further it was found that these athletes are mostly playing the following sports :-*

> *3x3 Wheelchair Basketball*

> *Lawn Bowls and Para Lawn Bowls*

> *Table Tennis and Para Table Tennis*

> *Triathlon and Para Triathlon*

****

**Now lets check male vs female count overall**

In [10]:
fig,ax=plt.subplots(figsize=(12,6))
plt.pie(x=players['GENDER'].value_counts(),labels=['Male','Female'],labeldistance=0.85,autopct='%1.1f%%',pctdistance=0.5)


centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax.axis('equal')  
plt.title('Overall Male vs Female Paticipation',fontsize=15)
fig.tight_layout()

**Conclusion**

*Male participants slightly edge out female participants for overall participation*

****

**On that note lets find out participation of Male vs Female athletes for each sport**

In [11]:
fig=px.histogram(y=players['SPORT'].sort_values(),color=players['GENDER'],color_discrete_sequence=['teal','palevioletred'],height=600,width=800).update_yaxes(categoryorder='total ascending')

fig.show()

**Conclusion**


*We have 3 sports only for Female athletes and 1 where only Male athletes participate*

****

# 

# EDA On Medal Winners Dataset

**Issues**

*There are some issues with the names of athletes in both datasets.Therefore some inaccurate results can be seen*

*Some players have been assigned medal when they have not won any*
> *for example :- Kenya have been assigned one extra bronze medal*

In [12]:
medals=pd.read_csv('../input/commonwealth-games-2022/commonwealth games 2022 - players won medals in cwg games 2022.csv')
medals=medals.replace(['G','S','B'],['Gold','Silver','Bronze'])
medal_tot=pd.get_dummies(medals['MEDAL'])
medal_tally=pd.concat([medals,medal_tot],axis=1)
medal_tally['Total']=medal_tally['Gold'] + medal_tally['Bronze'] + medal_tally['Silver']

**Lets find out the medal tally for each country**

**NOTE**
> ***if AUSTRALIA wins in 3 X 3 basketball then the dataset will show 3 medals againts their name***

> ***But according to the rules, a win in team sport gets assigned only 1 medal to the teams name. so essentially we need to drop the duplicates to get a proper medal tally***

In [13]:
newmedal_tally=medal_tally.drop_duplicates(subset=['TEAM','SPORT','EVENT','MEDAL','CONTINENT'])
newmedaltally_teams=newmedal_tally.groupby('TEAM')[['Gold','Silver','Bronze','Total']].sum().sort_values('Gold',ascending=False).reset_index()
newmedaltally_teams.head(5)
fig = px.histogram(y= newmedal_tally['TEAM'].sort_values(),color=newmedal_tally['MEDAL'],color_discrete_sequence=["dimgrey","saddlebrown","goldenrod",],height=1000,width=800,title='Total Medal Tally').update_yaxes(categoryorder='total ascending')
fig.show()


****

**Lets find the top 20 athletes with most medals**

In [14]:
ind=medal_tally.groupby(['ATHLETE NAME','MEDAL'])['Total'].sum().reset_index().sort_values('Total',ascending=False).head(20)
individual=medal_tally.merge(ind,on='ATHLETE NAME',how='inner')
newindividual=individual.sort_values(['Total_y'],ascending=False)
fig = px.histogram(y= newindividual['ATHLETE NAME'],color=newindividual['MEDAL_x'],color_discrete_sequence=["goldenrod","dimgrey","saddlebrown"],height=1000,width=800,title='Individual Medal Tally').update_yaxes(categoryorder='total ascending')
fig.show()

**Fact**
> *Swimmer Emma McKeon became the most decorated athlete in Commonwealth Games history winning eight medals*

****

# 

**Next we will find medal winners by gender and see distribution of medal winners by age**

*So first we will merge the two datasets*
> *The problem is that the first dataset names differ from that of the second and so we have some issues*

In [15]:
players['ATHLETE NAME\t'] = players['ATHLETE NAME\t'].str.replace('.', '')
players['ATHLETE NAME\t'] = players['ATHLETE NAME\t'].str.replace(' ', '')
medal_tally['ATHLETE NAME'] = medal_tally['ATHLETE NAME'].str.replace('.', '')
medal_tally['ATHLETE NAME'] = medal_tally['ATHLETE NAME'].str.replace(' ', '')
medal_tally['ATHLETE NAME'] = medal_tally['ATHLETE NAME'].str.replace('null', '')
Totalmedal_tally=medal_tally.merge(players[['GENDER','ATHLETE NAME\t','AGE']],left_on='ATHLETE NAME',right_on='ATHLETE NAME\t',how='inner')
Totalmedal_tally.drop('ATHLETE NAME\t',axis=1,inplace=True)
Totalmedal_tally['Gold']=Totalmedal_tally['Gold'].astype('int')
Totalmedal_tally['Silver']=Totalmedal_tally['Silver'].astype('int')
Totalmedal_tally['Bronze']=Totalmedal_tally['Bronze'].astype('int')
Totalmedal_tally['Total']=Totalmedal_tally['Total'].astype('int')

Totalmedal_tallyMale=Totalmedal_tally[Totalmedal_tally['GENDER'] == 'Male']
Totalmedal_tallyFemale=Totalmedal_tally[Totalmedal_tally['GENDER'] == 'Female']

**Create plot to show the distribution of Age of medal winnes for male athletes**

In [16]:
import plotly.figure_factory as ff
x1=Totalmedal_tallyMale['AGE']
x2=Totalmedal_tallyMale[Totalmedal_tallyMale['Gold']== 1]['AGE']
x3=Totalmedal_tallyMale[Totalmedal_tallyMale['Silver']== 1]['AGE']
x4=Totalmedal_tallyMale[Totalmedal_tallyMale['Bronze']== 1]['AGE']
fig = ff.create_distplot([x1, x2, x3, x4], ['Overall Age', 'Gold Medalist', 'Silver Medalist', 'Bronze Medalist'],show_hist=False, show_rug=False)
fig.show()

**Create plot to show the distribution of Age of medal winnes for female athletes**

In [17]:
import plotly.figure_factory as ff
x1=Totalmedal_tallyFemale['AGE']
x2=Totalmedal_tallyFemale[Totalmedal_tallyFemale['Gold']== 1]['AGE']
x3=Totalmedal_tallyFemale[Totalmedal_tallyFemale['Silver']== 1]['AGE']
x4=Totalmedal_tallyFemale[Totalmedal_tallyFemale['Bronze']== 1]['AGE']
fig = ff.create_distplot([x1, x2, x3, x4], ['Overall Age', 'Gold Medalist', 'Silver Medalist', 'Bronze Medalist'],show_hist=False, show_rug=False)
fig.show()

**Conclusion**

*Peak age of Medal winners is between 23-26*

*For mens sport*
> *For silver medalist the peak is between 23-24 years and for gold,bronze the peak is between 24-26 years*

*For womens sport*
> *we see that peak age for gold medal winners is between 25 and 28*
> *for Bronze medal the peak age is between 19 and 23*

****

# 

**So the last thing we will do is create a heatmap to show which country perrformed better at which sport**

In [18]:
Totalmedal_tally=Totalmedal_tally.drop_duplicates(subset=['TEAM','SPORT','EVENT','MEDAL','CONTINENT'])

fig,ax=plt.subplots(figsize=(16,10))
sns.heatmap(Totalmedal_tally.pivot_table(index='SPORT', columns='TEAM', values='Total', aggfunc='count').fillna(0).astype(int),annot=True)
plt.title('How Do Countries Perform Across Different Sports ?',fontsize=15)
fig.tight_layout()

# Thank You For Reading